In [1]:
# =====================================================
# 🏠 House Prices - XGBoost
# =====================================================
import pandas as pd

# XGBoost
from xgboost import XGBRegressor
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
import joblib
import matplotlib.pyplot as plt
from pathlib import Path

BASE = Path.cwd().parent   
# =====================================================
# ⚙️ 0. carregamento dos preprocessador 
# =====================================================
temp = joblib.load(BASE /'src'/'preprocess_house_prices_v1.joblib')
preprocessador=temp['preprocessador']
colnull_train = temp['colnull_train']

# =====================================================
# 📁 1. Leitura dos dados & Separação das bases
# =====================================================

DATA_DIR = BASE / "data" / "processed"
X_train = pd.read_csv(DATA_DIR / "X_train_final.csv")
X_test  = pd.read_csv(DATA_DIR / "X_test_final.csv")
y_train = pd.read_csv(DATA_DIR / "y_train_final.csv")
y_test  = pd.read_csv(DATA_DIR / "y_test_final.csv")

# =====================================================
#  🤖 4.Modelos
# =====================================================
model_xg0    = XGBRegressor( objective='reg:squarederror',random_state=42,n_jobs=-1 ) 
XGB0         = pipe_models(model_xg0 , preprocessador)

# 1. Valida a estabilidade 
r20 = valida(X_train, y_train, model=XGB0, N=7)

# 2. Testa a performance 
XGB0.fit(X_train, y_train)
y_pred = XGB0.predict(X_test)
res0 = metricas_model(y_test, y_pred, 'XGBoost 0')

Validação cruzada realizada!


In [26]:
# calcular importances
xgboost_model = XGB0.named_steps['model']
importances = xgboost_model.get_booster().get_score(importance_type='gain')

# IMPORTANTE: Ajustar o pré-processador primeiro
preprocessador.fit(X_train)  # Esta linha é crucial!

# Agora pode obter os nomes das features
feature_names = preprocessador.get_feature_names_out()

print(f"Número de features após pré-processamento: {len(feature_names)}")
print(f"Primeiras 10 features: {feature_names[:10]}")

# Criar mapeamento
feature_mapping = {f'f{i}': feature_names[i] for i in range(len(feature_names))}

# Converter para DataFrame com nomes reais
feat_imp = pd.DataFrame(list(importances.items()), columns=['feature_index', 'gain'])
feat_imp['feature'] = feat_imp['feature_index'].map(feature_mapping)
feat_imp = feat_imp.sort_values(by='gain', ascending=False)

print("\nTop 20 Features mais importantes:")
print(feat_imp[['feature', 'gain']].head(20))

Número de features após pré-processamento: 233
Primeiras 10 features: ['MSZoning_FV' 'MSZoning_RH' 'MSZoning_RL' 'MSZoning_RM' 'Street_Pave'
 'LotShape_IR2' 'LotShape_IR3' 'LotShape_Reg' 'LandContour_HLS'
 'LandContour_Low']

Top 20 Features mais importantes:
                  feature      gain
148           OverallQual  2.969193
169            GarageCars  1.858259
113          CentralAir_Y  0.265090
167            Fireplaces  0.252109
160             GrLivArea  0.204977
123        Functional_Sev  0.200222
137         GarageCond_TA  0.186017
129     GarageType_Detchd  0.169017
92            BsmtQual_Gd  0.104711
149           OverallCond  0.100912
3             MSZoning_RM  0.087136
156           TotalBsmtSF  0.080832
153            BsmtFinSF1  0.070283
119        KitchenQual_TA  0.066148
150             YearBuilt  0.065197
18   Neighborhood_ClearCr  0.059861
125     GarageType_Attchd  0.055156
164              HalfBath  0.050465
86           ExterCond_TA  0.049697
157              1st

In [25]:
# calcular importances
xgboost_model = XGB0.named_steps['model']
importances = xgboost_model.get_booster().get_score(importance_type='gain')

# Obter o pré-processador DO PIPELINE AJUSTADO
# Primeiro, descubra qual step é o pré-processador
for step_name, step_obj in XGB0.named_steps.items():
    if step_name != 'model':  # O pré-processador é o outro step
        preprocessor_fitted = step_obj
        break

# Agora extrair nomes (está fitted porque o pipeline foi ajustado)
feature_names = preprocessor_fitted.get_feature_names_out()

# Criar mapeamento
feature_mapping = {f'f{i}': feature_names[i] for i in range(len(feature_names))}

# Converter para DataFrame com nomes reais
feat_imp = pd.DataFrame(list(importances.items()), columns=['feature_index', 'gain'])
feat_imp['feature'] = feat_imp['feature_index'].map(feature_mapping)
feat_imp = feat_imp.sort_values(by='gain', ascending=False)

print(feat_imp[['feature', 'gain']].head(20))

                  feature      gain
148           OverallQual  2.969193
169            GarageCars  1.858259
113          CentralAir_Y  0.265090
167            Fireplaces  0.252109
160             GrLivArea  0.204977
123        Functional_Sev  0.200222
137         GarageCond_TA  0.186017
129     GarageType_Detchd  0.169017
92            BsmtQual_Gd  0.104711
149           OverallCond  0.100912
3             MSZoning_RM  0.087136
156           TotalBsmtSF  0.080832
153            BsmtFinSF1  0.070283
119        KitchenQual_TA  0.066148
150             YearBuilt  0.065197
18   Neighborhood_ClearCr  0.059861
125     GarageType_Attchd  0.055156
164              HalfBath  0.050465
86           ExterCond_TA  0.049697
157              1stFlrSF  0.039539


In [ ]:
# from scipy import stats

# def avaliar_distribuicoes(
#     df,
#     alpha=0.05,
#     lim_discreto=20,
#     max_shapiro=5000
# ):
#     """
#     Avalia tipo de variável e forma da distribuição para cada coluna do DataFrame.

#     Parâmetros
#     ----------
#     df : pd.DataFrame
#     alpha : float
#         Nível de significância para testes de normalidade
#     lim_discreto : int
#         Número máximo de valores únicos para considerar variável discreta
#     max_shapiro : int
#         Tamanho máximo da amostra para aplicar Shapiro-Wilk

#     Retorna
#     -------
#     pd.DataFrame com diagnóstico estatístico por feature
#     """

#     resultados = []

#     for col in df.columns:
#         s = df[col].dropna()
#         n = len(s)

#         if n == 0:
#             continue

#         info = {
#             "feature": col,
#             "n": n,
#             "dtype": str(df[col].dtype),
#             "n_unicos": s.nunique()
#         }

#         # ==============================
#         # 1️⃣ Tipo da variável
#         # ==============================
#         if pd.api.types.is_datetime64_any_dtype(s):
#             tipo = "data"
#             distribuicao = "temporal"

#         elif pd.api.types.is_bool_dtype(s):
#             tipo = "binária"
#             distribuicao = "binária"

#         elif pd.api.types.is_numeric_dtype(s):

#             if info["n_unicos"] <= 2:
#                 tipo = "binária"
#                 distribuicao = "binária"

#             elif info["n_unicos"] <= lim_discreto:
#                 tipo = "numérica discreta"
#             else:
#                 tipo = "numérica contínua"

#         else:
#             tipo = "categórica"
#             distribuicao = "categórica"

#         info["tipo"] = tipo

#         # ==============================
#         # 2️⃣ Estatísticas (numéricas)
#         # ==============================
#         if pd.api.types.is_numeric_dtype(s) and tipo != "binária":

#             mean = s.mean()
#             median = s.median()
#             std = s.std()
#             skew = stats.skew(s)
#             kurt = stats.kurtosis(s)

#             info.update({
#                 "media": mean,
#                 "mediana": median,
#                 "std": std,
#                 "skewness": skew,
#                 "kurtosis": kurt
#             })

#             # ==============================
#             # 3️⃣ Testes de normalidade
#             # ==============================
#             p_shapiro = np.nan
#             p_dagostino = np.nan

#             if n <= max_shapiro:
#                 p_shapiro = stats.shapiro(s.sample(n=min(n, max_shapiro)))[1]

#             if n > 20:
#                 p_dagostino = stats.normaltest(s)[1]

#             info["p_shapiro"] = p_shapiro
#             info["p_dagostino"] = p_dagostino

#             normal = (
#                 (not np.isnan(p_shapiro) and p_shapiro > alpha) or
#                 (not np.isnan(p_dagostino) and p_dagostino > alpha)
#             )

#             # ==============================
#             # 4️⃣ Classificação da distribuição
#             # ==============================
#             if normal and abs(skew) < 0.5:
#                 distribuicao = "aprox. normal"

#             elif skew > 1:
#                 distribuicao = "assimétrica à direita"

#             elif skew < -1:
#                 distribuicao = "assimétrica à esquerda"

#             elif abs(skew) < 0.5 and kurt < -1:
#                 distribuicao = "uniforme-like"

#             elif kurt > 3:
#                 distribuicao = "cauda pesada"

#             else:
#                 distribuicao = "não normal"

#         info["distribuicao"] = distribuicao

#         # ==============================
#         # 5️⃣ Recomendações
#         # ==============================
#         if tipo == "numérica contínua":
#             if distribuicao in ["assimétrica à direita", "cauda pesada"]:
#                 rec = "Avaliar log / Box-Cox / Yeo-Johnson"
#             elif distribuicao == "aprox. normal":
#                 rec = "Métodos paramétricos OK"
#             else:
#                 rec = "Avaliar métodos robustos ou não paramétricos"
#         elif tipo == "categórica":
#             rec = "Codificação (One-Hot / Ordinal)"
#         else:
#             rec = "Análise específica"

#         info["recomendacao"] = rec

#         resultados.append(info)

#     return pd.DataFrame(resultados)
# resumo = avaliar_distribuicoes(df)
# resumo.sort_values("distribuicao").head(60)

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy import stats
# from scipy.stats import shapiro, normaltest, kstest, anderson
# import warnings
# warnings.filterwarnings('ignore')

# def analisar_distribuicoes(df, amostras_grandes=5000, alpha=0.05):
#     """
#     Analisa as características e distribuições das variáveis de um DataFrame.
    
#     Parâmetros:
#     -----------
#     df : pandas DataFrame
#         DataFrame a ser analisado
#     amostras_grandes : int
#         Limite para considerar amostras grandes (default: 5000)
#     alpha : float
#         Nível de significância para testes estatísticos (default: 0.05)
    
#     Retorna:
#     --------
#     dict : Dicionário com análises completas de cada variável
#     """
    
#     resultados = {
#         'resumo_geral': {},
#         'variaveis_numericas': {},
#         'variaveis_categoricas': {},
#         'recomendacoes': []
#     }
    
#     # 1. Análise inicial do DataFrame
#     resultados['resumo_geral'] = {
#         'total_linhas': len(df),
#         'total_colunas': len(df.columns),
#         'tipos_dados': df.dtypes.value_counts().to_dict(),
#         'valores_nulos_porcentagem': (df.isnull().sum() / len(df) * 100).to_dict(),
#         'valores_unicos_por_coluna': df.nunique().to_dict()
#     }
    
#     # 2. Separar variáveis por tipo
#     variaveis_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
#     variaveis_categoricas = df.select_dtypes(exclude=[np.number]).columns.tolist()
    
#     # 3. Analisar variáveis numéricas
#     for coluna in variaveis_numericas:
#         dados = df[coluna].dropna()
        
#         # Estatísticas descritivas
#         estatisticas = {
#             'tipo': 'numérica',
#             'contagem': len(dados),
#             'nulos': df[coluna].isnull().sum(),
#             'media': dados.mean(),
#             'mediana': dados.median(),
#             'desvio_padrao': dados.std(),
#             'variancia': dados.var(),
#             'min': dados.min(),
#             'max': dados.max(),
#             'q1': dados.quantile(0.25),
#             'q3': dados.quantile(0.75),
#             'iqr': dados.quantile(0.75) - dados.quantile(0.25),
#             'assimetria': dados.skew(),
#             'curtose': dados.kurtosis(),
#             'cv': (dados.std() / dados.mean() * 100) if dados.mean() != 0 else np.nan
#         }
        
#         # Classificação da assimetria
#         skew_val = estatisticas['assimetria']
#         if abs(skew_val) < 0.5:
#             estatisticas['classificacao_assimetria'] = 'simétrica'
#         elif abs(skew_val) < 1:
#             estatisticas['classificacao_assimetria'] = 'moderadamente assimétrica'
#         else:
#             estatisticas['classificacao_assimetria'] = 'altamente assimétrica'
        
#         # Testes de normalidade
#         normalidade = {}
        
#         if len(dados) >= 3:  # Mínimo para alguns testes
#             # Shapiro-Wilk (recomendado para n < 5000)
#             if len(dados) < 5000:
#                 try:
#                     stat, p = shapiro(dados)
#                     normalidade['shapiro_wilk'] = {
#                         'estatistica': stat,
#                         'p_valor': p,
#                         'normal': p > alpha
#                     }
#                 except:
#                     pass
            
#             # D'Agostino's K² test (funciona para amostras maiores)
#             try:
#                 stat, p = normaltest(dados)
#                 normalidade['dagostino_k2'] = {
#                     'estatistica': stat,
#                     'p_valor': p,
#                     'normal': p > alpha
#                 }
#             except:
#                 pass
            
#             # Teste de Kolmogorov-Smirnov
#             try:
#                 # Comparar com distribuição normal com mesma média e desvio
#                 param = stats.norm.fit(dados)
#                 stat, p = kstest(dados, 'norm', args=param)
#                 normalidade['kolmogorov_smirnov'] = {
#                     'estatistica': stat,
#                     'p_valor': p,
#                     'normal': p > alpha
#                 }
#             except:
#                 pass
            
#             # Teste de Anderson-Darling
#             try:
#                 resultado = anderson(dados, dist='norm')
#                 normalidade['anderson_darling'] = {
#                     'estatistica': resultado.statistic,
#                     'valores_criticos': resultado.critical_values.tolist(),
#                     'niveis_significancia': resultado.significance_level.tolist()
#                 }
#             except:
#                 pass
        
#         estatisticas['testes_normalidade'] = normalidade
        
#         # Tentativa de identificar distribuição
#         distribuicao_identificada = identificar_distribuicao(dados, alpha)
#         estatisticas['distribuicao_identificada'] = distribuicao_identificada
        
#         # Identificação de outliers usando IQR
#         Q1 = estatisticas['q1']
#         Q3 = estatisticas['q3']
#         IQR = estatisticas['iqr']
#         limite_inferior = Q1 - 1.5 * IQR
#         limite_superior = Q3 + 1.5 * IQR
#         outliers = dados[(dados < limite_inferior) | (dados > limite_superior)]
        
#         estatisticas['outliers_iqr'] = {
#             'limite_inferior': limite_inferior,
#             'limite_superior': limite_superior,
#             'quantidade': len(outliers),
#             'porcentagem': (len(outliers) / len(dados)) * 100 if len(dados) > 0 else 0
#         }
        
#         resultados['variaveis_numericas'][coluna] = estatisticas
    
#     # 4. Analisar variáveis categóricas
#     for coluna in variaveis_categoricas:
#         dados = df[coluna].dropna()
        
#         estatisticas = {
#             'tipo': str(df[coluna].dtype),
#             'contagem': len(dados),
#             'nulos': df[coluna].isnull().sum(),
#             'valores_unicos': dados.nunique(),
#             'moda': dados.mode().iloc[0] if not dados.empty else None,
#             'frequencia_moda': dados.value_counts().iloc[0] if not dados.empty else 0,
#             'entropia': calcular_entropia(dados),
#             'distribuicao_categorias': dados.value_counts(normalize=True).to_dict()
#         }
        
#         # Classificação do tipo de variável categórica
#         if dados.nunique() == 2:
#             estatisticas['tipo_categorica'] = 'binária'
#         elif dados.nunique() <= 10:
#             estatisticas['tipo_categorica'] = 'nominal_pequena'
#         elif dados.nunique() <= 50:
#             estatisticas['tipo_categorica'] = 'nominal_grande'
#         else:
#             estatisticas['tipo_categorica'] = 'textual/alta_cardinalidade'
        
#         resultados['variaveis_categoricas'][coluna] = estatisticas
    
#     # 5. Gerar recomendações
#     gerar_recomendacoes(resultados)
    
#     return resultados


# def identificar_distribuicao(dados, alpha=0.05):
#     """
#     Tenta identificar a distribuição dos dados.
#     """
#     if len(dados) < 30:
#         return {'distribuicao': 'amostra_pequena', 'confianca': 'baixa'}
    
#     resultados = []
    
#     # Testar distribuição normal
#     try:
#         _, p_normal = normaltest(dados)
#         resultados.append(('normal', p_normal))
#     except:
#         pass
    
#     # Testar distribuição log-normal
#     try:
#         dados_positivos = dados[dados > 0]
#         if len(dados_positivos) > 30:
#             dados_log = np.log(dados_positivos)
#             _, p_lognormal = normaltest(dados_log)
#             resultados.append(('log_normal', p_lognormal))
#     except:
#         pass
    
#     # Testar distribuição exponencial
#     try:
#         param = stats.expon.fit(dados)
#         stat, p_expon = kstest(dados, 'expon', args=param)
#         resultados.append(('exponencial', p_expon))
#     except:
#         pass
    
#     # Verificar simetria e curtose para inferência
#     skewness = stats.skew(dados)
#     kurt = stats.kurtosis(dados)
    
#     # Classificação baseada em estatísticas descritivas
#     if abs(skewness) < 0.5 and abs(kurt) < 0.5:
#         inferencia = 'simétrica_mesocúrtica (similar à normal)'
#     elif skewness > 1:
#         inferencia = 'assimétrica_positiva'
#     elif skewness < -1:
#         inferencia = 'assimétrica_negativa'
#     elif kurt > 1:
#         inferencia = 'leptocúrtica (picos agudos)'
#     elif kurt < -1:
#         inferencia = 'platicúrtica (achatada)'
#     else:
#         inferencia = 'distribuição_regular'
    
#     # Escolher a melhor distribuição baseada nos testes
#     if resultados:
#         melhor_dist = max(resultados, key=lambda x: x[1])
#         distribuicao = melhor_dist[0] if melhor_dist[1] > alpha else 'desconhecida'
#         confianca = 'alta' if melhor_dist[1] > 0.1 else 'moderada'
#     else:
#         distribuicao = 'desconhecida'
#         confianca = 'baixa'
    
#     return {
#         'distribuicao_teste': distribuicao,
#         'inferencia_descritiva': inferencia,
#         'assimetria': skewness,
#         'curtose': kurt,
#         'confianca': confianca,
#         'resultados_testes': resultados
#     }


# def calcular_entropia(dados):
#     """
#     Calcula a entropia de Shannon para variáveis categóricas.
#     """
#     if len(dados) == 0:
#         return 0
    
#     contagens = dados.value_counts()
#     proporcoes = contagens / len(dados)
#     entropia = -np.sum(proporcoes * np.log2(proporcoes))
    
#     return entropia


# def gerar_recomendacoes(resultados):
#     """
#     Gera recomendações baseadas na análise.
#     """
#     recomendacoes = []
    
#     # Analisar variáveis numéricas
#     for coluna, info in resultados['variaveis_numericas'].items():
#         # Verificar normalidade
#         normal = any(teste.get('normal', False) 
#                     for teste in info.get('testes_normalidade', {}).values())
        
#         if not normal:
#             recomendacoes.append(
#                 f"Variável '{coluna}': Não normal - considerar transformações (log, box-cox) "
#                 f"ou usar testes não paramétricos"
#             )
        
#         # Verificar outliers
#         outliers_pct = info['outliers_iqr']['porcentagem']
#         if outliers_pct > 5:
#             recomendacoes.append(
#                 f"Variável '{coluna}': {outliers_pct:.1f}% outliers detectados - "
#                 f"verificar se são erros ou dados válidos"
#             )
        
#         # Verificar assimetria
#         if abs(info['assimetria']) > 1:
#             recomendacoes.append(
#                 f"Variável '{coluna}': Altamente assimétrica (skew={info['assimetria']:.2f}) - "
#                 f"considerar transformações"
#             )
    
#     # Analisar variáveis categóricas
#     for coluna, info in resultados['variaveis_categoricas'].items():
#         if info['valores_unicos'] > 50:
#             recomendacoes.append(
#                 f"Variável '{coluna}': Alta cardinalidade ({info['valores_unicos']} valores únicos) - "
#                 f"considerar agrupamento ou técnicas específicas"
#             )
    
#     # Verificar dados faltantes
#     for coluna, pct in resultados['resumo_geral']['valores_nulos_porcentagem'].items():
#         if pct > 20:
#             recomendacoes.append(
#                 f"Variável '{coluna}': {pct:.1f}% dados faltantes - "
#                 f"avaliar impacto e estratégia de imputação"
#             )
    
#     resultados['recomendacoes'] = recomendacoes


# def gerar_relatorio(resultados, formato='texto'):
#     """
#     Gera um relatório formatado da análise.
    
#     Parâmetros:
#     -----------
#     resultados : dict
#         Resultados da função analisar_distribuicoes
#     formato : str
#         'texto' ou 'dataframe' para formato de saída
#     """
#     if formato == 'texto':
#         # Resumo geral
#         print("=" * 80)
#         print("ANÁLISE DE DISTRIBUIÇÕES - RELATÓRIO")
#         print("=" * 80)
        
#         resumo = resultados['resumo_geral']
#         print(f"\n1. RESUMO GERAL:")
#         print(f"   Total de observações: {resumo['total_linhas']}")
#         print(f"   Total de variáveis: {resumo['total_colunas']}")
#         print(f"   Tipos de dados: {resumo['tipos_dados']}")
        
#         print(f"\n2. VARIÁVEIS NUMÉRICAS:")
#         for coluna, info in resultados['variaveis_numericas'].items():
#             print(f"\n   {coluna}:")
#             print(f"   - Tipo: {info['tipo']}")
#             print(f"   - Distribuição: {info['distribuicao_identificada']['distribuicao_teste']}")
#             print(f"   - Assimetria: {info['assimetria']:.3f} ({info['classificacao_assimetria']})")
#             print(f"   - Outliers: {info['outliers_iqr']['quantidade']} "
#                   f"({info['outliers_iqr']['porcentagem']:.1f}%)")
        
#         print(f"\n3. VARIÁVEIS CATEGÓRICAS:")
#         for coluna, info in resultados['variaveis_categoricas'].items():
#             print(f"\n   {coluna}:")
#             print(f"   - Tipo: {info['tipo_categorica']}")
#             print(f"   - Valores únicos: {info['valores_unicos']}")
#             print(f"   - Entropia: {info['entropia']:.3f}")
        
#         print(f"\n4. RECOMENDAÇÕES:")
#         for i, rec in enumerate(resultados['recomendacoes'], 1):
#             print(f"   {i}. {rec}")
        
#         print("\n" + "=" * 80)
#         print("FIM DO RELATÓRIO")
#         print("=" * 80)
    
#     elif formato == 'dataframe':
#         # Criar DataFrames resumidos
#         df_numericas = pd.DataFrame.from_dict(
#             resultados['variaveis_numericas'], 
#             orient='index'
#         )
        
#         df_categoricas = pd.DataFrame.from_dict(
#             resultados['variaveis_categoricas'], 
#             orient='index'
#         )
        
#         return {
#             'numericas': df_numericas,
#             'categoricas': df_categoricas,
#             'recomendacoes': resultados['recomendacoes']
#         }


# # Função auxiliar para visualização
# def plotar_distribuicoes(df, variaveis=None, figsize=(15, 10)):
#     """
#     Plota distribuições das variáveis.
#     """
#     if variaveis is None:
#         variaveis = df.columns
    
#     n_variaveis = len(variaveis)
#     n_cols = 3
#     n_rows = (n_variaveis + n_cols - 1) // n_cols
    
#     fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
#     axes = axes.flatten()
    
#     for idx, coluna in enumerate(variaveis):
#         ax = axes[idx]
#         dados = df[coluna].dropna()
        
#         if pd.api.types.is_numeric_dtype(df[coluna]):
#             # Plot para variáveis numéricas
#             ax.hist(dados, bins='auto', alpha=0.7, edgecolor='black')
#             ax.axvline(dados.mean(), color='red', linestyle='--', label=f'Média: {dados.mean():.2f}')
#             ax.axvline(dados.median(), color='green', linestyle='--', 
#                       label=f'Mediana: {dados.median():.2f}')
#             ax.set_title(f'{coluna}\n(skew={dados.skew():.2f}, kurt={dados.kurtosis():.2f})')
#             ax.legend()
#             ax.set_xlabel('Valor')
#             ax.set_ylabel('Frequência')
#         else:
#             # Plot para variáveis categóricas
#             top_categorias = dados.value_counts().head(10)
#             ax.bar(top_categorias.index.astype(str), top_categorias.values)
#             ax.set_title(f'{coluna}\n({len(dados.unique())} categorias)')
#             ax.set_xlabel('Categoria')
#             ax.set_ylabel('Contagem')
#             plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
    
#     # Ocultar eixos vazios
#     for idx in range(len(variaveis), len(axes)):
#         axes[idx].set_visible(False)
    
#     plt.tight_layout()
#     plt.show()


# # Exemplo de uso
# if __name__ == "__main__":
#     # Criar DataFrame de exemplo
   

#     # Executar análise
#     resultados = analisar_distribuicoes(df)
    
#     # Gerar relatório
#     gerar_relatorio(resultados, formato='texto')
    
#     # Visualizar distribuições
#     plotar_distribuicoes(df)
    
#     # Obter resultados em DataFrames
#     dfs_resultados = gerar_relatorio(resultados, formato='dataframe')
#     print("\nResumo numérico:")
#     print(dfs_resultados['numericas'][['media', 'assimetria', 'outliers_iqr']].head())

In [ ]:



# #columns
# # Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
# #        'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
# #        'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
# #        'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
# #        'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrArea', 'ExterQual',
# #        'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
# #        'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
# #        'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
# #        '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
# #        'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
# #        'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageType',
# #        'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
# #        'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
# #        'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
# #        'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'SalePrice'],
# #       dtype='object')

# import plotly.graph_objects as go
# import scipy.stats as st
# import numpy as np

# # 1. Preparar os dados
# y = df['GarageArea'] # <-- caixa de seleção

# # 2. Cálculos Estatísticos 
# params_johnson = st.johnsonsu.fit(y)
# params_norm = st.norm.fit(y)

# x_range = np.linspace(y.min(), y.max(), 200)
# pdf_johnson = st.johnsonsu.pdf(x_range, *params_johnson)
# pdf_norm = st.norm.pdf(x_range, *params_norm)

# # 3. Construção do Gráfico
# fig = go.Figure()

# # Histograma usando a primeira cor da sua paleta
# fig.add_trace(go.Histogram(
#     x=y,
#     histnorm='probability density',
#     name='Distribuição Real',
#     marker_color=color_palette21[0], 
#     opacity=0.6,
#     marker=dict(line=dict(width=1, color='white')) )
# )

# # Linha de Ajuste Johnson SU (usando um tom de laranja da sua paleta para destaque)
# fig.add_trace(go.Scatter(
#     x=x_range, y=pdf_johnson,
#     mode='lines',
#     name='Ajuste Johnson SU',
#     line=dict(color=color_palette21[18], width=4) # Laranja forte
# ))

# # Linha de Ajuste Normal (usando um tom mais claro para comparação)
# fig.add_trace(go.Scatter(
#     x=x_range, y=pdf_norm,
#     mode='lines',
#     name='Ajuste Normal (Gaussiana)',
#     line=dict(color=color_palette21[10], width=3, dash='dash')
# ))

# # 4. Layout
# fig.update_layout(
#     title='Análise de Distribuição: SalePrice',
#     xaxis_title='SalePrice',
#     yaxis_title='Densidade',
#     template='plotly_white',
#     legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
#     width=1000,
#     height=720
# )

# fig.show()